## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-13-22-53-19 +0000,nyt,U.S. to Name Palestinian Committee to Run Gaza,https://www.nytimes.com/2026/01/13/world/middl...
1,2026-01-13-22-52-00 +0000,wsj,Trump Tells Iranian Protesters ‘Help Is On Its...,https://www.wsj.com/world/middle-east/trump-ca...
2,2026-01-13-22-50-49 +0000,bbc,"More than 2,000 people reported killed at Iran...",https://www.bbc.com/news/articles/cr57vmznmd9o...
3,2026-01-13-22-50-00 +0000,wsj,Former New York City Mayor Eric Adams’s NYC To...,https://www.wsj.com/us-news/former-new-york-ci...
4,2026-01-13-22-49-42 +0000,nypost,"I’m living through Iran’s bloodthirsty, murder...",https://nypost.com/2026/01/13/world-news/im-li...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2361/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,61
13,iran,33
106,ice,17
253,fed,17
16,new,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
77,2026-01-13-21-07-00 +0000,wsj,President Trump said the economy was poised fo...,https://www.wsj.com/politics/policy/trump-pred...,150
225,2026-01-13-14-47-00 +0000,wsj,Some Trump administration officials and allies...,https://www.wsj.com/politics/policy/trump-alli...,135
350,2026-01-13-02-38-00 +0000,wsj,White House Weighs Iran’s Nuclear-Talks Offer ...,https://www.wsj.com/world/middle-east/trump-ir...,122
228,2026-01-13-14-35-20 +0000,nyt,Trump Says He Will Impose Tariffs on Iran’s Tr...,https://www.nytimes.com/2026/01/13/world/middl...,121
359,2026-01-13-01-44-20 +0000,cbc,"U.S. tells its citizens to leave Iran, as Trum...",https://www.cbc.ca/news/world/iran-protests-us...,117


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
77,150,2026-01-13-21-07-00 +0000,wsj,President Trump said the economy was poised fo...,https://www.wsj.com/politics/policy/trump-pred...
338,80,2026-01-13-03-47-00 +0000,wsj,Senate Republicans criticized the Justice Depa...,https://www.wsj.com/politics/policy/gop-senato...
29,66,2026-01-13-22-19-10 +0000,startribune,Supreme Court seems likely to uphold state ban...,https://www.startribune.com/supreme-court-take...
91,64,2026-01-13-20-40-15 +0000,nyt,"As Iran’s Government Tries to Quell Protests, ...",https://www.nytimes.com/2026/01/13/world/middl...
110,59,2026-01-13-20-07-43 +0000,nypost,New police dashcam video refutes claims that T...,https://nypost.com/2026/01/13/us-news/new-poli...
138,52,2026-01-13-18-32-52 +0000,nyt,"Before White House Meeting, Greenland’s Leader...",https://www.nytimes.com/2026/01/13/us/politics...
346,51,2026-01-13-03-06-52 +0000,nypost,Hakeem Jeffries calls for Kristi Noem ouster a...,https://nypost.com/2026/01/12/us-news/hakeem-j...
18,46,2026-01-13-22-37-28 +0000,nypost,Eric Adams tells heckler ‘go f–k yourself’ as ...,https://nypost.com/2026/01/13/us-news/eric-ada...
84,45,2026-01-13-20-51-30 +0000,cbc,"As Carney heads to China to talk trade, Ontari...",https://www.cbc.ca/news/canada/toronto/doug-fo...
94,45,2026-01-13-20-29-53 +0000,wapo,Prosecutors seek death for South Korea’s forme...,https://www.washingtonpost.com/world/2026/01/1...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
